In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Set display options

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line breaks


In [3]:
# Import the data
match_data = pd.read_csv("MatchesCombined_v3winners_training_data.csv")
match_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'MatchesCombined_v3winners_training_data.csv'

In [ ]:
# Combine "home_team_code" and "away_team_code" into one new column "team_code"

# Create a dataframe for home teams

home_match_data = match_data.copy()

home_match_data['team_code'] = home_match_data['home_team_code']

home_match_data['opponent_code'] = home_match_data['away_team_code']

home_match_data['home_or_away'] = 'home'

home_match_data.head()

In [ ]:
# Create a dataframe for away teams

away_match_data = match_data.copy()
away_match_data['team_code'] = away_match_data['away_team_code']
away_match_data['opponent_code'] = away_match_data['home_team_code']
away_match_data['home_or_away'] = 'away'

away_match_data.head()

In [ ]:
# Concatenate the two dataframes
combined_match_data = pd.concat([home_match_data, away_match_data], ignore_index=True)

combined_match_data.tail(25)

In [ ]:
# Sort the dataframe by 'team_code' alphabetically
combined_match_data = combined_match_data.sort_values(by='team_code')

# Reset the index after sorting
combined_match_data = combined_match_data.reset_index(drop=True)

# Display the tail of the new dataframe
combined_match_data.head(10)

In [ ]:
combined_match_data.columns

In [ ]:
# Drop columns

new_combined_match_data = combined_match_data.drop(columns=['winner', 
                                                        'winner score', 
                                                        'home_team_code', 
                                                        'away_team_code', 
                                                        'home_team', 
                                                        'away_team',
                                                        'home_penalty',
                                                        'away_penalty',
                                                        'stadium_name',
                                                        'stadium_name_media',
                                                        'stadium_name_official',
                                                        'stadium_name_event',
                                                        'stadium_name_sponsor',
                                                        'red_cards',
                                                        'game_referees',
                                                        'stadium_city',
                                                        'goals',
                                                        'date_time',
                                                        'utc_offset_hours',
                                                        'group_name',
                                                        'matchday_name',
                                                        'condition_humidity', 
                                                        'condition_pitch',
                                                        'condition_temperature',
                                                        'condition_weather', 
                                                        'condition_wind_speed', 
                                                        'status', 
                                                        'type', 
                                                        'round',
                                                        'round_mode',
                                                        'stadium_id', 
                                                        'stadium_country_code',
                                                        'stadium_capacity',
                                                        'stadium_name_media',
                                                        'stadium_name_official', 
                                                        'stadium_name_event', 
                                                        'stadium_name_sponsor','winner.1',
                                                        'winner_reason', 
                                                        'year',
                                                        'penalties_missed',
                                                        'penalties',
                                                        'home_score_total',
                                                        'away_score_total'])

new_combined_match_data.columns

In [ ]:
def move_column(df, column_name, new_position):
    cols = df.columns.tolist()
    cols.insert(new_position, cols.pop(cols.index(column_name)))
    return df[cols]

# Move column 'D' to be the second column
adjusted_match_data_df = move_column(new_combined_match_data, 'team_code', 1)

adjusted_match_data_df.head(25)

In [ ]:
# Delete the first 4 rows

adjusted_match_data_drop_rows_df = adjusted_match_data_df.drop(index=adjusted_match_data_df.index[:4])

adjusted_match_data_drop_rows_df = adjusted_match_data_drop_rows_df.reset_index(drop=True)

adjusted_match_data_drop_rows_df

In [ ]:
# Adjust the column names if necessary
new_team_data = adjusted_match_data_drop_rows_df.rename(columns={
    "home XG": "team_xG",
    "home odds to win": "team_odds",
    "away XG": "opponent_xG",
    "away odds to win": "opponent_odds",
    "home_score": "team_score",
    "away_score": "opponent_score",
    "home_penalty": "team_penalty",
    "away_penalty": "opponent_penalty",
    "Target - 1 means home team will be in the top 4": "target_top_4"
})

new_team_data

In [ ]:
# Delete the last 8 rows
new_team_data_row_delete = new_team_data.drop(index=new_team_data.index[-8:])

# Reset the index after deleting rows
new_team_data_row_delete = new_team_data_row_delete.reset_index(drop=True)

# Display the modified dataframe
new_team_data_row_delete


In [ ]:
# Strip any leading/trailing spaces and convert to lowercase
new_team_data_row_delete['home_or_away'] = new_team_data_row_delete['home_or_away'].str.strip().str.lower()

# Change home_or_away value from string to int 

new_team_data_row_delete['home_or_away'] = new_team_data_row_delete['home_or_away'].map({'home': 1, 'away': 0})

new_team_data_row_delete

In [ ]:
# Convert the 'id_match', 'opponent_odds', 'team_score', 'opponent_score', 'match_attendance', and 'target_top_4' columns from float to int

# Columns to convert
columns_to_convert = ['id_match', 'opponent_odds', 'team_score', 'opponent_score', 'match_attendance', 'target_top_4']

# Handle NaN values if any (e.g., fill with 0)
new_team_data_row_delete[columns_to_convert] = new_team_data_row_delete[columns_to_convert].fillna(0)

# Convert the specified columns from float to int
new_team_data_row_delete[columns_to_convert] = new_team_data_row_delete[columns_to_convert].astype(int)

new_team_data_row_delete

In [ ]:
# Define the new order of columns

euro_team_data = ['id_match', 
                  'date', 
                  'source', 
                  'home_or_away', 
                  'team_code', 
                  'team_xG', 
                  'team_odds', 
                  'opponent_code', 
                  'opponent_xG', 
                  'opponent_odds', 
                  'team_score', 
                  'opponent_score', 
                  'match_attendance', 
                  'stadium_latitude', 
                  'stadium_longitude', 
                  'stadium_pitch_length', 
                  'stadium_pitch_width', 
                  'target_top_4']

# Create a new dataframe with columns in the new order
euro_team_data_df = new_team_data_row_delete[euro_team_data]

euro_team_data_df


In [ ]:
# Count the number of 0s in the match_attendance column
count_zeros = euro_team_data_df['match_attendance'].value_counts().get(0, 0)

count_zeros

In [ ]:
# Calculate the mean of match_attendance excluding 0 values
mean_attendance = euro_team_data_df.loc[euro_team_data_df['match_attendance'] != 0, 'match_attendance'].mean()

# Replace 0 values with the mean attendance using .loc
euro_team_data_df.loc[:, 'match_attendance'] = euro_team_data_df['match_attendance'].replace(0, mean_attendance)

# Convert match_attendance from float to int using .loc
euro_team_data_df['match_attendance'] = euro_team_data_df['match_attendance'].astype(int)

euro_team_data_df


In [ ]:
# Find where the team is listed as "home_or_away", and where they are listed as away switch the "team_xG" and "opponent_xG"

# Find rows where home_or_away is 0
away_matches = df[df['home_or_away'] == 0]

# Switch team_xG and opponent_xG for away matches
df.loc[away_matches.index, ['team_xG', 'opponent_xG']] = df.loc[away_matches.index, ['opponent_xG', 'team_xG']].values

# Switch team_odds and opponent_odds for away matches
df.loc[away_matches.index, ['team_odds', 'opponent_odds']] = df.loc[away_matches.index, ['opponent_odds', 'team_odds']].values

# Switch team_score and opponent_score for away matches
df.loc[away_matches.index, ['team_score', 'opponent_score']] = df.loc[away_matches.index, ['opponent_score', 'team_score']].values

import ace_tools as tools; tools.display_dataframe_to_user(name="Corrected Euro Team Data", dataframe=df)

# Display the corrected dataframe
df.head()


In [ ]:
# # get dummies on 'team_code', 'opponent_code', 'source'

# # Identify the categorical columns
# categorical_columns = ['source', 'team_code', 'opponent_code']

# # Use get_dummies to one-hot encode these columns
# team_data_df_encoded = pd.get_dummies(new_team_data_row_delete, columns=categorical_columns)

# team_data_df_encoded


In [21]:
# euro_team_data_df.to_csv('euro_team_data.csv', index=False)


In [ ]:

#standard scaling on the final data